<a href="https://colab.research.google.com/github/alessandronicolini/IncrementalLearning/blob/main/baselines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import project files files from github 


In [1]:
# upload work files from your git hub repository
import sys

!git clone https://github.com/alessandronicolini/IncrementalLearning.git # clone proj repository
!rm -rf IncrementalLearning/README.md 
!rm -rf IncrementalLearning/baselines.ipynb

path = 'IncrementalLearning/'
if path not in sys.path:
    sys.path.append('IncrementalLearning/')

!pip3 install import_ipynb

fatal: destination path 'IncrementalLearning' already exists and is not an empty directory.


import libraries


In [2]:
import torch
import os
import torch.nn as nn
from torch.utils.data import Subset, DataLoader
import torch.optim as optim
from torchvision import transforms
import numpy as np
import import_ipynb

# project classes --------------------------------------------------------------
from cifar100 import ilCIFAR100
# from baselines.resnet import resnet32
# from info_recorder.info_log import InfoLog
# from models_benchmark.benchmark import Benchmark

importing Jupyter notebook from cifar100.ipynb
Files already downloaded and verified
[60, 34, 84, 67, 85, 44, 18, 48, 1, 47, 61, 35, 82, 58, 76, 29, 71, 0, 79, 93, 56, 90, 20, 43, 26, 7, 73, 25, 9, 65, 95, 51, 11, 2, 74, 28, 96, 27, 99, 64, 70, 42, 62, 8, 98, 77, 39, 88, 10, 94, 3, 52, 68, 32, 5, 72, 38, 75, 69, 30, 40, 41, 24, 55, 91, 45, 12, 16, 22, 53, 63, 57, 31, 33, 21, 83, 49, 81, 59, 78, 97, 19, 46, 17, 36, 87, 6, 13, 14, 89, 80, 23, 86, 37, 54, 92, 50, 66, 15, 4]
0
1
2
3
4
5
6
7
8
9
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


Initial setup

In [3]:
# Define params

NUM_RUN = 2
SEEDS = [1,2,3]
NUM_EPOCHS = 5
BATCH_SIZE = 64
CLASSES_PER_BATCH = 10
DEVICE = 'cuda'

WEIGHT_DECAY = 1e-5
LR = 2
MOMENTUM = 0.9
MILESTONES = [49, 63]
GAMMA = 0.2

SAVINGS_DIR = "savings"
# Create savings folder
try:
    os.mkdir(SAVINGS_DIR)
except FileExistsError:
    pass

Define transforms

In [4]:
# Define transformations for training
train_transform = transforms.Compose([transforms.RandomCrop(32, padding=4),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Define transformations for evaluation
test_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))                                    
])

Prepare dataloders for each train, val test subset of the original dataset

In [6]:
# dataloaders dict initialization
dataloaders = {run:{batch:{'train':None, 'val':None, 'test':None} for batch in range(CLASSES_PER_BATCH)} for run in range(NUM_RUN)}

# make dataloaders
for run in range(NUM_RUN):
    
    # download dataset only on the first run
    download = True
    if run != 0:
        download = False
    
    original_training_set = ilCIFAR100(10,12)
    original_test_set = ilCIFAR100(CLASSES_PER_BATCH, train=False, seed=SEEDS[run])
    
    train_indices, val_indices = original_training_set.get_train_val(0.1)
    test_indices = original_training_set.get_batch_indexes()

    for class_batch in range(CLASSES_PER_BATCH):
        
        # test indices are the uninion of test indices of batches seen up to now
            

        # make subsets
        train_set = Subset(dataset=original_training_set, indices=train_indices[class_batch])
        val_set = Subset(dataset=original_training_set, indices=val_indices[class_batch])
        test_set = Subset(dataset=original_test_set, indices=test_indices[class_batch])

        # make dataloaders
        train_dataloader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
        val_dataloader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True)
        test_dataloader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)

        # add dataloaders to the dataloaders dict
        dataloaders[run][class_batch]['train'] = train_dataloader
        dataloaders[run][class_batch]['val'] = val_dataloader
        dataloaders[run][class_batch]['test'] = test_dataloader

Files already downloaded and verified
[60, 34, 84, 67, 85, 44, 18, 48, 1, 47, 61, 35, 82, 58, 76, 29, 71, 0, 79, 93, 56, 90, 20, 43, 26, 7, 73, 25, 9, 65, 95, 51, 11, 2, 74, 28, 96, 27, 99, 64, 70, 42, 62, 8, 98, 77, 39, 88, 10, 94, 3, 52, 68, 32, 5, 72, 38, 75, 69, 30, 40, 41, 24, 55, 91, 45, 12, 16, 22, 53, 63, 57, 31, 33, 21, 83, 49, 81, 59, 78, 97, 19, 46, 17, 36, 87, 6, 13, 14, 89, 80, 23, 86, 37, 54, 92, 50, 66, 15, 4]
Files already downloaded and verified
[17, 72, 97, 8, 32, 15, 63, 57, 60, 83, 48, 26, 12, 62, 3, 49, 55, 77, 0, 92, 34, 29, 75, 13, 40, 85, 2, 74, 69, 1, 89, 27, 54, 98, 28, 56, 93, 35, 14, 22, 61, 43, 59, 71, 78, 18, 70, 88, 86, 41, 6, 11, 82, 46, 67, 7, 21, 95, 68, 42, 87, 19, 45, 31, 47, 25, 73, 30, 94, 23, 79, 39, 76, 58, 5, 64, 99, 91, 52, 24, 33, 80, 16, 66, 90, 96, 50, 84, 36, 44, 81, 10, 9, 38, 20, 4, 51, 65, 37, 53]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'

FINE TUNING

In [7]:
for run in range(NUM_RUN):
    benchmark = Benchmark(
        num_epochs=NUM_EPOCHS, 
        dataloaders=dataloaders,
        batch_size=BATCH_SIZE,
        saving_folder="savings/fine_tuning")
    
    benchmark.set_criterion(nn.CrossEntropyLoss())
    benchmark.set_infoLog(InfoLog, run)
    benchmark.set_model(resnet32())

    for class_batch in range(NUM_CLASS_BATCH):
        parameters_to_optimize = benchmark.model.parameters()
        benchmark.set_optimizer(optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY))
        benchmark.set_scheduler(optim.lr_scheduler.MultiStepLR(benchmark.optimizer, milestones=MILESTONES, gamma=GAMMA))

        benchmark.do_class_batch(run, class_batch)

        benchmark.model.add_nodes(10) # forse è meglio metterlo all'inizio

NameError: ignored